## Libraries

In [ ]:
import sys
import os
import datetime as dt
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
from vega_datasets import data

In [ ]:
if not 'mainDir' in globals():
    mainDir = os.path.dirname(os.getcwd()) # Get parent dir: os.path.dirname()
print(mainDir)

In [ ]:
dir_input = os.path.join(mainDir,'data','raw')
dir_output = os.path.join(mainDir,'data','processed')

## Settings

In [ ]:
first_relevant_year = 1850

start_reference_period = 1961
end_reference_period = 1990

In [ ]:
list_relevant_iso = [
    'CAN',
    'USA',
    'NIC',
    'BRA',
    'ARG',
    'GRL',
    'ISL',
    'DEU',
    'ESP',
    'EGY',
    'MRT',
    'MDG',
    'ZAF',
    'COD',
    'SAU',
    'RUS',
    'CHN',
    'JPN',
    'IND',
    'IDN',
    'AUS',
    'NZL',
    #'ATF',
]

## Read data

### Read shapes

In [ ]:
gdf = gpd.read_file(os.path.join(dir_input,"countries.geojson"))
gdf.shape

In [ ]:
gdf = gdf.rename(columns={'ADMIN':'Country'})
if len(list_relevant_iso) > 0:
    gdf = gdf.loc[gdf['ISO_A3'].isin(list_relevant_iso)]
gdf.shape

In [ ]:
list_relevant_countries = gdf['Country'].unique().tolist()

### Read climate data

In [ ]:
df = pd.read_csv(os.path.join(dir_input,"GlobalLandTemperaturesByCountry.csv"))
df.shape

In [ ]:
df = df[['dt','AverageTemperature','Country']]

In [ ]:
df['dt'] = pd.to_datetime(df['dt'], format="%Y-%m-%d")
df = df.loc[(df['dt'].dt.year>=first_relevant_year)]
df['Year'] = df['dt'].dt.year
df['Month'] = df['dt'].dt.month
df.shape
df.head(3)

In [ ]:
country_names_dict = {
    'Antigua And Barbuda':'Antigua and Barbuda',
    'Bahamas':'The Bahamas',
    'Bosnia And Herzegovina':'Bosnia and Herzegovina',
    'Burma':'Myanmar',
    'Congo':'Republic of Congo',
    'Congo (Democratic Republic Of The)':'Democratic Republic of the Congo',
    "Côte D'Ivoire":'Ivory Coast',
    'Falkland Islands (Islas Malvinas)':'Falkland Islands',
    'Federated States Of Micronesia':'Federated States of Micronesia',
    'French Southern And Antarctic Lands':'French Southern and Antarctic Lands',
    'Heard Island And Mcdonald Islands':'Heard Island and McDonald Islands',
    'Hong Kong':'Hong Kong S.A.R.',
    'Isle Of Man':'Isle of Man',
    'Macau':'Macao S.A.R',
    'Palestina':'Palestine',
    'Saint Barthélemy':'Saint Barthelemy',
    'Saint Kitts And Nevis':'Saint Kitts and Nevis',
    'Saint Pierre And Miquelon':'Saint Pierre and Miquelon',
    'Saint Vincent And The Grenadines':'Saint Vincent and the Grenadines',
    'Sao Tome And Principe':'Sao Tome and Principe',
    'Serbia':'Republic of Serbia',
    'South Georgia And The South Sandwich Isla':'South Georgia and South Sandwich Islands',
    'Tanzania':'United Republic of Tanzania',
    'Timor Leste':'East Timor',
    'Trinidad And Tobago':'Trinidad and Tobago',
    'Turks And Caicas Islands':'Turks and Caicos Islands',
    'United States':'United States of America',
    'Virgin Islands':'United States Virgin Islands',
    'Åland':'Aland'
}

In [ ]:
for key in country_names_dict.keys():
    #print(key, country_names_dict[key])
    df.loc[df['Country']==key,'Country'] = country_names_dict[key]

In [ ]:
if len(list_relevant_countries) > 0:
    df = df.loc[df['Country'].isin(list_relevant_countries)]
df.shape

## Prepare data

### Heatmap data

In [ ]:
df_reference = df.loc[(df['Year']>=start_reference_period) & (df['Year']<=end_reference_period)]
df_reference = pd.DataFrame(df_reference.groupby(['Country','Month'])['AverageTemperature'].mean())
#df_reference

In [ ]:
country_counter = 1
for country in list_relevant_countries:
    if country_counter % 10 == 0:
        print(f"{country_counter} of {len(list_relevant_countries)} countries")
    for month in range(1,13):
        df.loc[
            (df['Country']==country) &
            (df['Month']==month),
            'AverageTemperature_ref'] = df.loc[
            (df['Country']==country) &
            (df['Month']==month),
            'AverageTemperature'] - df_reference.loc[(country,month),'AverageTemperature']
    country_counter += 1

### Map data (centroids)

In [ ]:
df_stats_countries = pd.DataFrame(df.loc[df['Year']>end_reference_period].groupby('Country')['AverageTemperature_ref'].mean()).reset_index()

In [ ]:
gdf = pd.merge(
    gdf,
    df_stats_countries,
    on='Country', how='left'
).rename(columns={'AverageTemperature_ref':'AverageTemperature_sinceRef'})
gdf['centroid'] = gdf.centroid
gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y
#gdf

In [ ]:
gdf.loc[gdf['ISO_A2']=='US',['longitude','latitude']] = [-98.5,39.5]
gdf.loc[gdf['ISO_A2']=='CA',['longitude','latitude']] = [-96.5,54.5]
#gdf

## Create dashboard

In [ ]:
alt.data_transformers.enable('default')
alt.data_transformers.disable_max_rows()

### Plot settings

In [ ]:
# Set time range
first_analysis_year = 1850
last_analysis_year = 2013

In [ ]:
# Set size
width_map = 700
height_map = 350
width_plots = width_map*1.5
height_plots = width_plots/((last_analysis_year-first_analysis_year)/12)

In [ ]:
# Set general colors
color_background = 'white' # '#FFFCF2'
color_titles = 'black' # '#4f4f4f'
color_labels = '#4f4f4f' # '#6e6e6e'

# Set colors background map
fill_sphere = '#D5F5FF' # '#7FD1D8'
stroke_color_graticule = 'white'
stroke_width_graticule = 1.0
fill_countries = '#DED9D8' # '#DED5D2' # 'lightgrey'
stroke_color_countries = '#978984' # 'grey'

# Set colors points
stroke_color_points = color_titles
color_points_selected = '#FF00E8'
color_points_not_selected = '#bbb1ae' # stroke_color_countries # '#2ECC71'
size_points_selected = 400
size_points_not_selected = 200
stroke_width_points_selected = 4
stroke_width_points_not_selected = 3

# Set colors heatmap
domain_heatmap = [-5,-.5,0,.5,5]
range_heatmap = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020']
color_background_heatmap = fill_countries # '#e0e0e0'

# Set colors trends
color_trend_selected = color_points_selected # '#FF5733'
color_trends_not_selected = color_points_not_selected # color_labels
stroke_width_trends_selected = 2.5
stroke_width_trends_not_selected = 1.25
opacity_trends_not_selected = 0.2
color_zero_line = 'grey' # '#008FFF'
stroke_width_zero_line = 1.2

In [ ]:
# Set selection
default_country = "Germany"
selector = alt.selection(type="single", fields=['Country'], init={"Country": default_country}, empty='none')

### Create annotations

In [ ]:
# Create title
title_text = ["Climate change is a global phenomenon"]
source_title = pd.DataFrame({'text': title_text})
title = alt.Chart(
    source_title
).mark_text(
    size=16,
    fontWeight='bold',
    align='center',
    color=color_titles,
    dx=width_plots/2,
).encode(
    text="text:N",
)
# Create credentials
credentials_text = ["Source: Berkeley Earth | Visualization: Lasse Scheele (@LasSchee)"]
source_credentials = pd.DataFrame({'text': credentials_text})
credentials = alt.Chart(
    source_credentials
).mark_text(
    size=11,
    align='right',
    dx=width_plots,
    color=color_labels
).encode(
    text="text:N",
)
credentials_text = ["Source: Berkeley Earth (berkeleyearth.org)"]
source_credentials = pd.DataFrame({'text': credentials_text})
credentials1 = alt.Chart(
    source_credentials
).mark_text(
    size=11,
    align='right',
    dx=width_plots,
    dy=-5,
    color=color_labels
).encode(
    text="text:N",
)
credentials_text = ["Visualization: Lasse Scheele (@LasSchee)"]
source_credentials = pd.DataFrame({'text': credentials_text})
credentials2 = alt.Chart(
    source_credentials
).mark_text(
    size=11,
    align='right',
    dx=width_plots,
    dy=10,
    color=color_labels
).encode(
    text="text:N",
)
#title + credentials
#title + credentials1 + credentials2

### Create map

In [ ]:
# Create basemap
source_sphere = alt.sphere()
source_graticule = alt.graticule()
source_map = alt.topo_feature(data.world_110m.url, 'countries')
background = alt.layer(
    alt.Chart(source_sphere).mark_geoshape(fill=fill_sphere),
    alt.Chart(source_graticule).mark_geoshape(stroke=stroke_color_graticule, strokeWidth=stroke_width_graticule),
    alt.Chart(source_map).mark_geoshape(fill=fill_countries, stroke=stroke_color_countries, strokeWidth=0.5),
).project(
    'naturalEarth1'
).properties(
    width=width_map, height=height_map,
    #title="Map of the available countries"
)
# Add points
source_points = gdf[['Country','longitude','latitude']]
color_points = alt.condition(selector,alt.value(color_points_selected),alt.value(color_points_not_selected))
size_points = alt.condition(selector,alt.value(size_points_selected),alt.value(size_points_not_selected))
strokeWidth_points = alt.condition(selector,alt.value(stroke_width_points_selected),alt.value(stroke_width_points_not_selected))
points = alt.Chart(
    source_points
).mark_circle(
    stroke=stroke_color_points,
    opacity=1.0
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=color_points,
    size=size_points,
    strokeWidth=strokeWidth_points,
    tooltip=['Country:N']
).add_selection(
    selector
)
#(background + points)

### Create line plots

In [ ]:
# Create trends
source_trend = df.loc[
    df['Year'].isin(list(range(first_analysis_year,last_analysis_year+1)))
].groupby(['Country','Year'])['AverageTemperature_ref'].mean().reset_index()
domain_trends = list_relevant_countries
range_trends = [color_trend_selected for x in list_relevant_countries]
color_trends = alt.condition(
    selector,
    alt.Color('Country:N', scale=alt.Scale(domain=domain_trends, range=range_trends), legend=None),
    alt.value(color_trends_not_selected))
strokeWidth_trends = alt.condition(selector,alt.value(stroke_width_trends_selected),alt.value(stroke_width_trends_not_selected))
opacity_trends = alt.condition(selector,alt.value(1),alt.value(opacity_trends_not_selected))
zero_line = alt.Chart(
    pd.DataFrame({'AverageTemperature_ref': [0]})
).mark_rule(
    color=color_zero_line,
    strokeWidth=stroke_width_zero_line
).encode(
    y='AverageTemperature_ref:Q'
)

trends = alt.Chart(
    source_trend,
    title=f'Average yearly temperature {first_analysis_year}-{last_analysis_year} (compared to the period {start_reference_period}-{end_reference_period})'
).mark_line(interpolate='basis'
).encode(
    x=alt.X('Year:O', sort=alt.EncodingSortField('Year', order='ascending')),
    y=alt.Y('AverageTemperature_ref:Q', title='°C'),
    color=color_trends,
    opacity=opacity_trends,
    strokeWidth=strokeWidth_trends,
    #size=alt.Size('AverageTemperature_ref:Q', legend=None),
    tooltip=[
        'Country:N','Year:Q',
        alt.Tooltip(
            'AverageTemperature_ref:Q',
            title=f'Average monthly temperature change compared to {start_reference_period}-{end_reference_period} (°C)',
            format='+.2f'
        ),
    ]
).add_selection(
    selector
).properties(
    width=width_plots,
    height=height_plots*1.4,
)
#(zero_line + trends)

### Create heatmap

In [ ]:
# Create heatmap
source_heatmap = df.loc[
    df['Year'].isin(list(range(first_analysis_year,last_analysis_year+1))),
    ['Year','Month','Country','AverageTemperature_ref']
]
heatmap_background = alt.Chart(
    pd.DataFrame(index=pd.MultiIndex.from_product([range(first_analysis_year,last_analysis_year+1), range(1,13)],names=['Year', 'Month'])).reset_index()
).mark_rect(
    color=color_background_heatmap,
    strokeWidth=0.5,
).encode(
    y=alt.Y('Month:O', sort=alt.EncodingSortField('Month', order='ascending')),
    x=alt.X('Year:O', sort=alt.EncodingSortField('Year', order='ascending')),
).properties(
    width=width_plots,
    height=height_plots,
)

heatmap = alt.Chart(
    source_heatmap,
    title=f'Average monthly temperature {first_analysis_year}-{last_analysis_year} (compared to the period {start_reference_period}-{end_reference_period})'
).mark_rect(
    #stroke='grey',
    #strokeWidth=0.5,
).encode(
    y=alt.Y('Month:O', sort=alt.EncodingSortField('Month', order='ascending')),
    x=alt.X('Year:O', sort=alt.EncodingSortField('Year', order='ascending')),
    color=alt.Color(
        'AverageTemperature_ref:Q',
        scale=alt.Scale(domain=domain_heatmap, range=range_heatmap),
        #scale=alt.Scale(type='sqrt', scheme="redyellowblue", order="descending"), # domain=[max_value, -max_value]
        title='Temperature difference (°C)'
    ),
    tooltip=[
        'Country:N','Year:O','Month:O',
        alt.Tooltip(
            'AverageTemperature_ref:Q',
            title=f'Average monthly temperature change compared to {start_reference_period}-{end_reference_period} (°C)',
            format='+.2f'),
    ]
).add_selection(
    selector
).transform_filter(
    selector
).properties(
    width=width_plots,
    height=height_plots,
)
#(heatmap_background + heatmap)

### Create final chart

In [ ]:
chart = alt.vconcat(
    #title + credentials,
    title + credentials1 + credentials2,
    (background + points),
    (zero_line + trends),
    (heatmap_background + heatmap),
    #credentials,
    center=True
).configure(
    background=color_background,
).configure_axis(
    titleColor=color_titles,
    labelColor=color_labels,
).configure_legend(
    orient='bottom',
    titleColor=color_titles,
    labelColor=color_labels
).configure_title(
    color=color_titles,
).configure_view(
    stroke=None
)
chart

In [ ]:
file_name = f"countries_centroids_tavg_{first_analysis_year}-{last_analysis_year}_ref{start_reference_period}-{end_reference_period}"
print(file_name)

In [ ]:
chart.save(os.path.join(mainDir,'docs',file_name+'.html'))
chart.save(os.path.join(mainDir,'plots',file_name+'.png'), scale_factor=2.5)